# Load medical images

This notebook introduces how to easily load different formats of medical images in MONAI and execute many additional operations.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/master/modules/load_medical_images.ipynb)

## Setup environment

In [1]:
%pip install -q "monai[itk, nibabel, pillow]"

Note: you may need to restart the kernel to use updated packages.


## Setup imports

In [1]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import shutil
import numpy as np
import itk
from PIL import Image
import tempfile
from monai.data import ITKReader, NibabelReader, PILReader
from monai.transforms import LoadImage, LoadImaged, AddChanneld, Resized, ToTensord, Compose
from monai.config import print_config

print_config()

MONAI version: 0.4.0
Numpy version: 1.19.1
Pytorch version: 1.7.0a0+7036e91
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 0563a4467fa602feca92d91c7f47261868d171a1

Optional dependencies:
Pytorch Ignite version: 0.4.2
Nibabel version: 3.2.1
scikit-image version: 0.15.0
Pillow version: 7.0.0
Tensorboard version: 2.2.0
gdown version: 3.12.2
TorchVision version: 0.8.0a0
ITK version: 5.1.0
tqdm version: 4.54.1
lmdb version: 1.0.0
psutil version: 5.7.2

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Load Nifti image with default image reader
MONAI automatically choose readers based on the supported suffixes and in below order:
- User specified reader at runtime when call this loader.
- Registered readers from the latest to the first in list.
- Default readers: (nii, nii.gz -> NibabelReader), (png, jpg, bmp -> PILReader),
  (npz, npy -> NumpyReader), (others -> ITKReader).

In [2]:
# generate 3D test images
tempdir = tempfile.mkdtemp()
test_image = np.random.rand(64, 128, 96)
filename = os.path.join(tempdir, "test_image.nii.gz")
itk_np_view = itk.image_view_from_array(test_image)
itk.imwrite(itk_np_view, filename)
data, meta = LoadImage()(filename)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

image data shape:(96, 128, 64)
meta data:{'sizeof_hdr': array(348, dtype=int32), 'extents': array(0, dtype=int32), 'session_error': array(0, dtype=int16), 'dim_info': array(0, dtype=uint8), 'dim': array([  3,  96, 128,  64,   1,   1,   1,   1], dtype=int16), 'intent_p1': array(0., dtype=float32), 'intent_p2': array(0., dtype=float32), 'intent_p3': array(0., dtype=float32), 'intent_code': array(0, dtype=int16), 'datatype': array(64, dtype=int16), 'bitpix': array(64, dtype=int16), 'slice_start': array(0, dtype=int16), 'pixdim': array([1., 1., 1., 1., 0., 0., 0., 0.], dtype=float32), 'vox_offset': array(0., dtype=float32), 'scl_slope': array(nan, dtype=float32), 'scl_inter': array(nan, dtype=float32), 'slice_end': array(0, dtype=int16), 'slice_code': array(0, dtype=uint8), 'xyzt_units': array(2, dtype=uint8), 'cal_max': array(0., dtype=float32), 'cal_min': array(0., dtype=float32), 'slice_duration': array(0., dtype=float32), 'toffset': array(0., dtype=float32), 'glmax': array(0, dtype=int

## Load a list of Nifti images and stack as 1 training item
Loading a list of files, stack them together and add a new dimension as first dimension.

And use the meta data of the first image to represent the stacked result.

In [3]:
filenames = ["test_image.nii.gz", "test_image2.nii.gz", "test_image3.nii.gz"]
for i, name in enumerate(filenames):
    filenames[i] = os.path.join(tempdir, name)
    itk_np_view = itk.image_view_from_array(test_image)
    itk.imwrite(itk_np_view, filenames[i])
data, meta = LoadImage()(filenames)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

image data shape:(3, 96, 128, 64)
meta data:{'sizeof_hdr': array(348, dtype=int32), 'extents': array(0, dtype=int32), 'session_error': array(0, dtype=int16), 'dim_info': array(0, dtype=uint8), 'dim': array([  3,  96, 128,  64,   1,   1,   1,   1], dtype=int16), 'intent_p1': array(0., dtype=float32), 'intent_p2': array(0., dtype=float32), 'intent_p3': array(0., dtype=float32), 'intent_code': array(0, dtype=int16), 'datatype': array(64, dtype=int16), 'bitpix': array(64, dtype=int16), 'slice_start': array(0, dtype=int16), 'pixdim': array([1., 1., 1., 1., 0., 0., 0., 0.], dtype=float32), 'vox_offset': array(0., dtype=float32), 'scl_slope': array(nan, dtype=float32), 'scl_inter': array(nan, dtype=float32), 'slice_end': array(0, dtype=int16), 'slice_code': array(0, dtype=uint8), 'xyzt_units': array(2, dtype=uint8), 'cal_max': array(0., dtype=float32), 'cal_min': array(0., dtype=float32), 'slice_duration': array(0., dtype=float32), 'toffset': array(0., dtype=float32), 'glmax': array(0, dtype=

## Load 3D image in DICOM format

In [4]:
filename = os.path.join(tempdir, "test_image.dcm")
dcm_image = np.random.randint(256, size=(64, 128, 96)).astype(np.uint8())
itk_np_view = itk.image_view_from_array(dcm_image)
itk.imwrite(itk_np_view, filename)
data, meta = LoadImage()(filename)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

image data shape:(64, 128, 96)
meta data:{'0008|0016': '1.2.840.10008.5.1.4.1.1.7.2', '0008|0018': '1.2.826.0.1.3680043.2.1125.1.48121911194701381427308384534193298', '0008|0020': '20201203', '0008|0030': '113120.042364 ', '0008|0050': '', '0008|0060': 'OT', '0008|0090': '', '0010|0010': '', '0010|0020': '', '0010|0030': '', '0010|0040': '', '0020|000d': '1.2.826.0.1.3680043.2.1125.1.67827102273467545876342130323471784', '0020|000e': '1.2.826.0.1.3680043.2.1125.1.16685311693546245602348836305872770', '0020|0010': '', '0020|0011': '', '0020|0013': '', '0020|0052': '1.2.826.0.1.3680043.2.1125.1.87673039291229035414733513900009302', '0028|0002': '1', '0028|0004': 'MONOCHROME2 ', '0028|0008': '64', '0028|0009': '(5200,9230)', '0028|0010': '128', '0028|0011': '96', '0028|0100': '8', '0028|0101': '8', '0028|0102': '7', '0028|0103': '0', '0028|1052': '0 ', '0028|1053': '1 ', '0028|1054': 'US', 'origin': array([0., 0., 0.]), 'spacing': array([1., 1., 1.]), 'direction': array([[1., 0., 0.],
   

## Load a list of DICOM images and stack as 1 training item
Loading a list of files, stack them together and add a new dimension as first dimension.

And use the meta data of the first image to represent the stacked result.

In [5]:
filenames = ["test_image.dcm", "test_image2.dcm", "test_image3.dcm"]
for i, name in enumerate(filenames):
    filenames[i] = os.path.join(tempdir, name)
    itk_np_view = itk.image_view_from_array(dcm_image)
    itk.imwrite(itk_np_view, filenames[i])
data, meta = LoadImage()(filenames)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

image data shape:(3, 64, 128, 96)
meta data:{'0008|0016': '1.2.840.10008.5.1.4.1.1.7.2', '0008|0018': '1.2.826.0.1.3680043.2.1125.1.95816205894474468453474236341374700', '0008|0020': '20201203', '0008|0030': '113131.022706 ', '0008|0050': '', '0008|0060': 'OT', '0008|0090': '', '0010|0010': '', '0010|0020': '', '0010|0030': '', '0010|0040': '', '0020|000d': '1.2.826.0.1.3680043.2.1125.1.68883679054578817264222174002573084', '0020|000e': '1.2.826.0.1.3680043.2.1125.1.16326677399477134127555792500385604', '0020|0010': '', '0020|0011': '', '0020|0013': '', '0020|0052': '1.2.826.0.1.3680043.2.1125.1.23377996774628184458694935299107085', '0028|0002': '1', '0028|0004': 'MONOCHROME2 ', '0028|0008': '64', '0028|0009': '(5200,9230)', '0028|0010': '128', '0028|0011': '96', '0028|0100': '8', '0028|0101': '8', '0028|0102': '7', '0028|0103': '0', '0028|1052': '0 ', '0028|1053': '1 ', '0028|1054': 'US', 'origin': array([0., 0., 0.]), 'spacing': array([1., 1., 1.]), 'direction': array([[1., 0., 0.],


## Load 2D image in PNG format

In [6]:
test_image = np.random.randint(0, 256, size=[128, 256])
filename = os.path.join(tempdir, "test_image.png")
Image.fromarray(test_image.astype("uint8")).save(filename)
data, meta = LoadImage()(filename)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

image data shape:(128, 256)
meta data:{'format': 'PNG', 'mode': 'L', 'width': 256, 'height': 128, 'spatial_shape': array([256, 128]), 'filename_or_obj': '/tmp/tmpht5hrp9_/test_image.png'}


## Load image with specified image reader
And we can set additional parameters for the image readers, for example, set `c_order_axis_indexing=True` for `ITKReader`, this parameter will pass to ITK `read()` function later.

In [7]:
loader = LoadImage()
loader.register(ITKReader())
data, meta = loader(filename)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

image data shape:(128, 256)
meta data:{'origin': array([0., 0.]), 'spacing': array([1., 1.]), 'direction': array([[1., 0.],
       [0., 1.]]), 'original_affine': array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), 'affine': array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), 'spatial_shape': array([128, 256]), 'filename_or_obj': '/tmp/tmpht5hrp9_/test_image.png'}


## Load image and execute additional operations
Some image readers can support additional operations after reading the image from file.

For example, we can set a converter for PILReader: `PILReader(converter=lambda image: image.convert("LA"))`.

In [8]:
loader = LoadImage(PILReader(converter=lambda image: image.convert("LA")))
data, meta = loader(filename)

print(f"image data shape:{data.shape}")
print(f"meta data:{meta}")

image data shape:(128, 256, 2)
meta data:{'format': None, 'mode': 'LA', 'width': 256, 'height': 128, 'spatial_shape': array([256, 128]), 'filename_or_obj': '/tmp/tmpht5hrp9_/test_image.png'}


## Connect `LoadImage` with other transforms
It's very easy to connect `LoadImage` transform with other transforms to construct a transform chain.

In [9]:
transform = Compose([
    LoadImaged(keys="image"),
    AddChanneld(keys="image"),
    Resized(keys="image", spatial_size=[64, 64]),
    ToTensord("image"),
])
test_data = {"image": filename}
result = transform(test_data)
print(f"image data shape:{result['image'].shape}")
print(f"meta data:{result['image_meta_dict']}")

image data shape:torch.Size([1, 64, 64])
meta data:{'format': 'PNG', 'mode': 'L', 'width': 256, 'height': 128, 'spatial_shape': array([256, 128]), 'filename_or_obj': '/tmp/tmpht5hrp9_/test_image.png'}


## Cleanup data directory

Remove directory if a temporary was used.

In [10]:
shutil.rmtree(tempdir)